# Guineapig image generator using GAN

Creating a guinea pig image generator using GANs would require substantial programming skills, especially in deep learning frameworks like TensorFlow or PyTorch. Here, I'll provide a high-level Python code structure using TensorFlow and Keras to give you an idea of how you might implement such a program:

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Conv2D, Conv2DTranspose, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist  # You'll replace this with your guinea pig dataset
import matplotlib.pyplot as plt

In [2]:
LATENT_DIM = 100
IMAGE_SHAPE = (64, 64, 3)  # Adjust dimensions according to your dataset
BATCH_SIZE = 64
EPOCHS = 10000
GENERATE_SAMPLES = 16

In [3]:
def build_generator():
    model = Sequential([
        Dense(8 * 8 * 256, input_dim=LATENT_DIM),
        Reshape((8, 8, 256)),
        Conv2DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu'),
        Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu'),
        Conv2DTranspose(3, kernel_size=3, strides=2, padding='same', activation='sigmoid')  # Adjust output channels
    ])
    return model

In [4]:
def build_discriminator():
    model = Sequential([
        Conv2D(64, kernel_size=3, strides=2, padding='same', activation='relu', input_shape=IMAGE_SHAPE),
        Conv2D(128, kernel_size=3, strides=2, padding='same', activation='relu'),
        Conv2D(256, kernel_size=3, strides=2, padding='same', activation='relu'),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

In [5]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential([
        generator,
        discriminator
    ])
    return model

In [6]:
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

C:\Users\Koushika\anaconda3\program\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Koushika\anaconda3\program\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
generator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

In [8]:
import os
import cv2

# Define a function to load images from a directory
def load_images_from_dir(directory, image_size=(64, 64)):
    images = []
    for filename in os.listdir(directory):
        img = cv2.imread(os.path.join(directory, filename))
        if img is not None:
            img = cv2.resize(img, image_size)
            images.append(img)
    return np.array(images)

# Load real images from your dataset directory
train_data_dir = 'C:/Users/Koushika/Downloads/guineapig.jpg'
real_images = load_images_from_dir(train_data_dir)

In [ ]:
# Compile discriminator model
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Training loop
for epoch in range(EPOCHS):
    # Train discriminator
    noise = np.random.normal(0, 1, (BATCH_SIZE, LATENT_DIM))
    generated_images = generator.predict(noise)
    real_images = load_images_from_dir(train_data_dir)  # Load real images from your dataset
    discriminator_loss_real = discriminator.train_on_batch(real_images, np.ones((len(real_images), 1)))
    discriminator_loss_fake = discriminator.train_on_batch(generated_images, np.zeros((BATCH_SIZE, 1)))
    discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)
    
    # Train generator
    noise = np.random.normal(0, 1, (BATCH_SIZE, LATENT_DIM))
    generator_loss = gan.train_on_batch(noise, np.ones((BATCH_SIZE, 1)))
    
    # Print progress
    print(f"Epoch {epoch}/{EPOCHS} -- D loss: {discriminator_loss} -- G loss: {generator_loss}")
    # Optionally, save generated images
    if epoch % 100 == 0:
        noise = np.random.normal(0, 1, (GENERATE_SAMPLES, LATENT_DIM))
        generated_images = generator.predict(noise)
        # Save or display generated_images


2/2 ━━━━━━━━━━━━━━━━━━━━ 29s 288ms/step


C:\Users\Koushika\anaconda3\program\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0/10000 -- D loss: 8.319717407226562 -- G loss: 0.7188280820846558
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 563ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Epoch 1/10000 -- D loss: 6.61937141418457 -- G loss: 0.7187550663948059
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Epoch 2/10000 -- D loss: 6.279334545135498 -- G loss: 0.7186762690544128
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Epoch 3/10000 -- D loss: 6.13362979888916 -- G loss: 0.7185814380645752
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Epoch 4/10000 -- D loss: 6.052703857421875 -- G loss: 0.7184903025627136
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Epoch 5/10000 -- D loss: 6.001221656799316 -- G loss: 0.7184009552001953
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Epoch 6/10000 -- D loss: 5.9655938148498535 -- G loss: 0.7183011174201965
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Epoch 7/10000 -- D loss: 5.939478874206543 -- G loss: 0.7182052135467529
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Epoch 8/10000 -- D loss: 5.919522762298584 -- G loss: 0.7181021571159